In [1]:
import sys
    # caution: path[0] is reserved for script path (or '' in REPL)
sys.path.insert(1, 'D:/books/python/0.   Fluent Python, 2nd Edition/example-code-2e/23-descriptor/')
sys.path

['D:\\books\\python\\0.   Fluent Python, 2nd Edition',
 'D:/books/python/0.   Fluent Python, 2nd Edition/example-code-2e/23-descriptor/',
 'C:\\Users\\lidan\\miniconda3\\python38.zip',
 'C:\\Users\\lidan\\miniconda3\\DLLs',
 'C:\\Users\\lidan\\miniconda3\\lib',
 'C:\\Users\\lidan\\miniconda3',
 '',
 'C:\\Users\\lidan\\AppData\\Roaming\\Python\\Python38\\site-packages',
 'C:\\Users\\lidan\\miniconda3\\lib\\site-packages',
 'C:\\Users\\lidan\\miniconda3\\lib\\site-packages\\magic_impute-2.0.4-py3.8.egg',
 'C:\\Users\\lidan\\miniconda3\\lib\\site-packages\\seqc-0.2.0-py3.8.egg',
 'C:\\Users\\lidan\\miniconda3\\lib\\site-packages\\weasyprint-56.1-py3.8.egg',
 'C:\\Users\\lidan\\miniconda3\\lib\\site-packages\\cairocffi-1.3.0-py3.8.egg',
 'C:\\Users\\lidan\\miniconda3\\lib\\site-packages\\win32',
 'C:\\Users\\lidan\\miniconda3\\lib\\site-packages\\win32\\lib',
 'C:\\Users\\lidan\\miniconda3\\lib\\site-packages\\Pythonwin']

In [2]:
"""

A line item for a bulk food order has description, weight and price fields::

    >>> raisins = LineItem('Golden raisins', 10, 6.95)
    >>> raisins.weight, raisins.description, raisins.price
    (10, 'Golden raisins', 6.95)

A ``subtotal`` method gives the total price for that line item::

    >>> raisins.subtotal()
    69.5

The weight of a ``LineItem`` must be greater than 0::

    >>> raisins.weight = -20
    Traceback (most recent call last):
        ...
    ValueError: weight must be > 0

Negative or 0 price is not acceptable either::

    >>> truffle = LineItem('White truffle', 100, 0)
    Traceback (most recent call last):
        ...
    ValueError: price must be > 0

No change was made::

    >>> raisins.weight
    10

"""


# tag::LINEITEM_QUANTITY_V3[]
class Quantity:  # <1>

    def __init__(self, storage_name):
        self.storage_name = storage_name  # <2>

    def __set__(self, instance, value):  # <3>
        if value > 0:
            instance.__dict__[self.storage_name] = value  # <4>
        else:
            msg = f'{self.storage_name} must be > 0'
            raise ValueError(msg)

    def __get__(self, instance, owner):  # <5>
        return instance.__dict__[self.storage_name]


# end::LINEITEM_QUANTITY_V3[]

# tag::LINEITEM_V3[]
class LineItem:
    weight = Quantity('weight')  # <1>
    price = Quantity('price')  # <2>

    def __init__(self, description, weight, price):  # <3>
        self.description = description
        self.weight = weight
        self.price = price

    def subtotal(self):
        return self.weight * self.price
# end::LINEITEM_V3[]


In [3]:
raisins = LineItem('Golden raisins', 10, 6.95)
raisins.weight, raisins.description, raisins.price

(10, 'Golden raisins', 6.95)

In [4]:
raisins.subtotal()

69.5

In [5]:
raisins.weight

10

In [7]:
raisins.weight = 20
raisins.subtotal()

139.0

In [9]:
raisins.weight, raisins.description, raisins.price

(20, 'Golden raisins', 6.95)

In [29]:
"""

A line item for a bulk food order has description, weight and price fields::

    >>> raisins = LineItem('Golden raisins', 10, 6.95)
    >>> raisins.weight, raisins.description, raisins.price
    (10, 'Golden raisins', 6.95)

A ``subtotal`` method gives the total price for that line item::

    >>> raisins.subtotal()
    69.5

The weight of a ``LineItem`` must be greater than 0::

    >>> raisins.weight = -20
    Traceback (most recent call last):
        ...
    ValueError: weight must be > 0

No change was made::

    >>> raisins.weight
    10

Negative or 0 price is not acceptable either::

    >>> truffle = LineItem('White truffle', 100, 0)
    Traceback (most recent call last):
        ...
    ValueError: price must be > 0

If the descriptor is accessed in the class, the descriptor object is
returned:

    >>> LineItem.weight  # doctest: +ELLIPSIS
    <bulkfood_v4.Quantity object at 0x...>
    >>> LineItem.weight.storage_name
    'weight'

"""


# tag::LINEITEM_V4[]
class Quantity:

    def __set_name__(self, owner, name):  # <1>
        self.storage_name = name          # <2>

    def __set__(self, instance, value):   # <3>
        if value > 0:
            instance.__dict__[self.storage_name] = value
        else:
            msg = f'{self.storage_name} must be > 0'
            raise ValueError(msg)

    # no __get__ needed  # <4>

class LineItem:
    weight = Quantity()  # <5>
    price = Quantity()

    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price

    def subtotal(self):
        return self.weight * self.price
# end::LINEITEM_V4[]


In [30]:
raisins = LineItem('Golden raisins', 10, 6.95)
raisins.weight, raisins.description, raisins.price

(10, 'Golden raisins', 6.95)

In [31]:
raisins.subtotal()

69.5

In [32]:
LineItem.weight  # doctest: +ELLIPSIS

In [33]:
LineItem.weight.storage_name

'weight'

In [34]:
LineItem.price.storage_name

'price'

In [35]:
Quantity()

In [21]:
# tag::MODEL_V5_VALIDATED_ABC[]
import abc

class Validated(abc.ABC):

    def __set_name__(self, owner, name):
        self.storage_name = name

    def __set__(self, instance, value):
        value = self.validate(self.storage_name, value)  # <1>
        instance.__dict__[self.storage_name] = value  # <2>

    @abc.abstractmethod
    def validate(self, name, value):  # <3>
        """return validated value or raise ValueError"""
# end::MODEL_V5_VALIDATED_ABC[]

# tag::MODEL_V5_VALIDATED_SUB[]
class Quantity(Validated):
    """a number greater than zero"""

    def validate(self, name, value):  # <1>
        if value <= 0:
            raise ValueError(f'{name} must be > 0')
        return value


class NonBlank(Validated):
    """a string with at least one non-space character"""

    def validate(self, name, value):
        value = value.strip()
        if not value:  # <2>
            raise ValueError(f'{name} cannot be blank')
        return value  # <3>
# end::MODEL_V5_VALIDATED_SUB[]


class LineItem:
    description = NonBlank()  # <2>
    weight = Quantity()
    price = Quantity()

    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price

    def subtotal(self):
        return self.weight * self.price
# end::LINEITEM_V5[]

In [22]:
raisins = LineItem('Golden raisins', 10, 6.95)
raisins.weight, raisins.description, raisins.price

(10, 'Golden raisins', 6.95)

In [23]:
raisins.subtotal()

69.5

In [24]:
LineItem.weight  # doctest: +ELLIPSIS

In [25]:
LineItem.weight.storage_name

'weight'

In [26]:
br_nuts = LineItem('Brazil Nuts', 10, 34.95)
br_nuts.description

'Brazil Nuts'

In [27]:
br_nuts.description = ' '

ValueError: description cannot be blank

In [36]:
"""
Overriding descriptor (a.k.a. data descriptor or enforced descriptor):

# tag::DESCR_KINDS_DEMO1[]

    >>> obj = Managed()  # <1>
    >>> obj.over  # <2>
    -> Overriding.__get__(<Overriding object>, <Managed object>, <class Managed>)
    >>> Managed.over  # <3>
    -> Overriding.__get__(<Overriding object>, None, <class Managed>)
    >>> obj.over = 7  # <4>
    -> Overriding.__set__(<Overriding object>, <Managed object>, 7)
    >>> obj.over  # <5>
    -> Overriding.__get__(<Overriding object>, <Managed object>, <class Managed>)
    >>> obj.__dict__['over'] = 8  # <6>
    >>> vars(obj)  # <7>
    {'over': 8}
    >>> obj.over  # <8>
    -> Overriding.__get__(<Overriding object>, <Managed object>, <class Managed>)

# end::DESCR_KINDS_DEMO1[]

Overriding descriptor without ``__get__``:

(these tests are reproduced below without +ELLIPSIS directives for inclusion in the book;
look for DESCR_KINDS_DEMO2)

    >>> obj.over_no_get  # doctest: +ELLIPSIS
    <descriptorkinds.OverridingNoGet object at 0x...>
    >>> Managed.over_no_get  # doctest: +ELLIPSIS
    <descriptorkinds.OverridingNoGet object at 0x...>
    >>> obj.over_no_get = 7
    -> OverridingNoGet.__set__(<OverridingNoGet object>, <Managed object>, 7)
    >>> obj.over_no_get  # doctest: +ELLIPSIS
    <descriptorkinds.OverridingNoGet object at 0x...>
    >>> obj.__dict__['over_no_get'] = 9
    >>> obj.over_no_get
    9
    >>> obj.over_no_get = 7
    -> OverridingNoGet.__set__(<OverridingNoGet object>, <Managed object>, 7)
    >>> obj.over_no_get
    9

Non-overriding descriptor (a.k.a. non-data descriptor or shadowable descriptor):

# tag::DESCR_KINDS_DEMO3[]

    >>> obj = Managed()
    >>> obj.non_over  # <1>
    -> NonOverriding.__get__(<NonOverriding object>, <Managed object>, <class Managed>)
    >>> obj.non_over = 7  # <2>
    >>> obj.non_over  # <3>
    7
    >>> Managed.non_over  # <4>
    -> NonOverriding.__get__(<NonOverriding object>, None, <class Managed>)
    >>> del obj.non_over  # <5>
    >>> obj.non_over  # <6>
    -> NonOverriding.__get__(<NonOverriding object>, <Managed object>, <class Managed>)

# end::DESCR_KINDS_DEMO3[]

No descriptor type survives being overwritten on the class itself:

# tag::DESCR_KINDS_DEMO4[]

    >>> obj = Managed()  # <1>
    >>> Managed.over = 1  # <2>
    >>> Managed.over_no_get = 2
    >>> Managed.non_over = 3
    >>> obj.over, obj.over_no_get, obj.non_over  # <3>
    (1, 2, 3)

# end::DESCR_KINDS_DEMO4[]

Methods are non-overriding descriptors:

    >>> obj.spam  # doctest: +ELLIPSIS
    <bound method Managed.spam of <descriptorkinds.Managed object at 0x...>>
    >>> Managed.spam  # doctest: +ELLIPSIS
    <function Managed.spam at 0x...>
    >>> obj.spam()
    -> Managed.spam(<Managed object>)
    >>> Managed.spam()
    Traceback (most recent call last):
      ...
    TypeError: Managed.spam() missing 1 required positional argument: 'self'
    >>> Managed.spam(obj)
    -> Managed.spam(<Managed object>)
    >>> Managed.spam.__get__(obj)  # doctest: +ELLIPSIS
    <bound method Managed.spam of <descriptorkinds.Managed object at 0x...>>
    >>> obj.spam.__func__ is Managed.spam
    True
    >>> obj.spam = 7
    >>> obj.spam
    7


"""

"""
NOTE: These tests are here because I can't add callouts after +ELLIPSIS
directives and if doctest runs them without +ELLIPSIS I get test failures.

# tag::DESCR_KINDS_DEMO2[]

    >>> obj.over_no_get  # <1>
    <__main__.OverridingNoGet object at 0x665bcc>
    >>> Managed.over_no_get  # <2>
    <__main__.OverridingNoGet object at 0x665bcc>
    >>> obj.over_no_get = 7  # <3>
    -> OverridingNoGet.__set__(<OverridingNoGet object>, <Managed object>, 7)
    >>> obj.over_no_get  # <4>
    <__main__.OverridingNoGet object at 0x665bcc>
    >>> obj.__dict__['over_no_get'] = 9  # <5>
    >>> obj.over_no_get  # <6>
    9
    >>> obj.over_no_get = 7  # <7>
    -> OverridingNoGet.__set__(<OverridingNoGet object>, <Managed object>, 7)
    >>> obj.over_no_get  # <8>
    9

# end::DESCR_KINDS_DEMO2[]

Methods are non-overriding descriptors:

# tag::DESCR_KINDS_DEMO5[]

    >>> obj = Managed()
    >>> obj.spam  # <1>
    <bound method Managed.spam of <descriptorkinds.Managed object at 0x74c80c>>
    >>> Managed.spam  # <2>
    <function Managed.spam at 0x734734>
    >>> obj.spam = 7  # <3>
    >>> obj.spam
    7

# end::DESCR_KINDS_DEMO5[]

"""

# tag::DESCR_KINDS[]

### auxiliary functions for display only ###

def cls_name(obj_or_cls):
    cls = type(obj_or_cls)
    if cls is type:
        cls = obj_or_cls
    return cls.__name__.split('.')[-1]

def display(obj):
    cls = type(obj)
    if cls is type:
        return f'<class {obj.__name__}>'
    elif cls in [type(None), int]:
        return repr(obj)
    else:
        return f'<{cls_name(obj)} object>'

def print_args(name, *args):
    pseudo_args = ', '.join(display(x) for x in args)
    print(f'-> {cls_name(args[0])}.__{name}__({pseudo_args})')


### essential classes for this example ###

class Overriding:  # <1>
    """a.k.a. data descriptor or enforced descriptor"""

    def __get__(self, instance, owner):
        print_args('get', self, instance, owner)  # <2>

    def __set__(self, instance, value):
        print_args('set', self, instance, value)


class OverridingNoGet:  # <3>
    """an overriding descriptor without ``__get__``"""

    def __set__(self, instance, value):
        print_args('set', self, instance, value)


class NonOverriding:  # <4>
    """a.k.a. non-data or shadowable descriptor"""

    def __get__(self, instance, owner):
        print_args('get', self, instance, owner)


class Managed:  # <5>
    over = Overriding()
    over_no_get = OverridingNoGet()
    non_over = NonOverriding()

    def spam(self):  # <6>
        print(f'-> Managed.spam({display(self)})')

# end::DESCR_KINDS[]


In [37]:
obj = Managed()  # <1>
obj.over  # <2>

-> Overriding.__get__(<Overriding object>, <Managed object>, <class Managed>)


In [38]:
Managed.over  # <3>

-> Overriding.__get__(<Overriding object>, None, <class Managed>)


In [39]:
obj.over = 7  # <4>

-> Overriding.__set__(<Overriding object>, <Managed object>, 7)


In [40]:
obj.over  # <5>

-> Overriding.__get__(<Overriding object>, <Managed object>, <class Managed>)


In [41]:
obj.__dict__['over'] = 8  # <6>
vars(obj)  # <7>

{'over': 8}

In [42]:
obj.over  # <8>

-> Overriding.__get__(<Overriding object>, <Managed object>, <class Managed>)


In [43]:
obj.over_no_get  # doctest: +ELLIPSIS

In [44]:
Managed.over_no_get  # doctest: +ELLIPSIS

In [45]:
obj.over_no_get = 7

-> OverridingNoGet.__set__(<OverridingNoGet object>, <Managed object>, 7)


In [46]:
obj.over_no_get  # doctest: +ELLIPSIS

In [47]:
obj.__dict__['over_no_get'] = 9
obj.over_no_get

9

In [48]:
obj.over_no_get = 7

-> OverridingNoGet.__set__(<OverridingNoGet object>, <Managed object>, 7)


In [49]:
obj.over_no_get

9

In [50]:
obj = Managed()
obj.non_over  # <1>

-> NonOverriding.__get__(<NonOverriding object>, <Managed object>, <class Managed>)


In [51]:
obj.non_over = 7  # <2>
obj.non_over  # <3>

7

In [52]:
Managed.non_over  # <4>

-> NonOverriding.__get__(<NonOverriding object>, None, <class Managed>)


In [53]:
del obj.non_over  # <5>
obj.non_over  # <6>

-> NonOverriding.__get__(<NonOverriding object>, <Managed object>, <class Managed>)


In [54]:
obj = Managed()  # <1>
Managed.over = 1  # <2>
Managed.over_no_get = 2
Managed.non_over = 3
obj.over, obj.over_no_get, obj.non_over  # <3>

(1, 2, 3)

In [55]:
obj.spam  # doctest: +ELLIPSIS

<bound method Managed.spam of <__main__.Managed object at 0x00000227E9BCEF70>>

In [56]:
Managed.spam  # doctest: +ELLIPSIS

<function __main__.Managed.spam(self)>

In [57]:
obj.spam()

-> Managed.spam(<Managed object>)


In [58]:
Managed.spam()

TypeError: spam() missing 1 required positional argument: 'self'

In [59]:
Managed.spam(obj)

-> Managed.spam(<Managed object>)


In [60]:
Managed.spam.__get__(obj)  # doctest: +ELLIPSIS

<bound method Managed.spam of <__main__.Managed object at 0x00000227E9BCEF70>>

In [61]:
obj.spam.__func__ is Managed.spam

True

In [62]:
obj.spam = 7
obj.spam

7

In [63]:
obj.over_no_get  # <1>

2

In [64]:
Managed.over_no_get  # <2>

2

In [66]:
obj.over_no_get = 7  # <3>
obj.over_no_get  # <4>

7

In [67]:
obj.__dict__['over_no_get'] = 9  # <5>
obj.over_no_get  # <6>

9

In [68]:
obj.over_no_get = 7  # <7>

In [69]:
obj.over_no_get  # <8>

7

In [70]:
obj = Managed()
obj.spam  # <1>

<bound method Managed.spam of <__main__.Managed object at 0x00000227E9C06A90>>

In [71]:
Managed.spam  # <2>

<function __main__.Managed.spam(self)>

In [72]:
obj.spam = 7  # <3>
obj.spam

7

In [73]:
"""
# tag::FUNC_DESCRIPTOR_DEMO[]

    >>> word = Text('forward')
    >>> word  # <1>
    Text('forward')
    >>> word.reverse()  # <2>
    Text('drawrof')
    >>> Text.reverse(Text('backward'))  # <3>
    Text('drawkcab')
    >>> type(Text.reverse), type(word.reverse)  # <4>
    (<class 'function'>, <class 'method'>)
    >>> list(map(Text.reverse, ['repaid', (10, 20, 30), Text('stressed')]))  # <5>
    ['diaper', (30, 20, 10), Text('desserts')]
    >>> Text.reverse.__get__(word)  # <6>
    <bound method Text.reverse of Text('forward')>
    >>> Text.reverse.__get__(None, Text)  # <7>
    <function Text.reverse at 0x101244e18>
    >>> word.reverse  # <8>
    <bound method Text.reverse of Text('forward')>
    >>> word.reverse.__self__  # <9>
    Text('forward')
    >>> word.reverse.__func__ is Text.reverse  # <10>
    True

# end::FUNC_DESCRIPTOR_DEMO[]
"""

# tag::FUNC_DESCRIPTOR_EX[]
import collections


class Text(collections.UserString):

    def __repr__(self):
        return 'Text({!r})'.format(self.data)

    def reverse(self):
        return self[::-1]

# end::FUNC_DESCRIPTOR_EX[]


In [74]:
word = Text('forward')
word  # <1>

Text('forward')

In [75]:
word.reverse()  # <2>

Text('drawrof')

In [76]:
Text.reverse(Text('backward'))  # <3>

Text('drawkcab')

In [77]:
type(Text.reverse), type(word.reverse)  # <4>

(function, method)

In [78]:
list(map(Text.reverse, ['repaid', (10, 20, 30), Text('stressed')]))  # <5>

['diaper', (30, 20, 10), Text('desserts')]

In [79]:
Text.reverse.__get__(word)  # <6>

<bound method Text.reverse of Text('forward')>

In [80]:
Text.reverse.__get__(None, Text)  # <7>

<function __main__.Text.reverse(self)>

In [81]:
word.reverse  # <8>

<bound method Text.reverse of Text('forward')>

In [82]:
word.reverse.__self__  # <9>

Text('forward')

In [83]:
word.reverse.__func__ is Text.reverse  # <10>

True

In [84]:
word.reverse.__call__

<method-wrapper '__call__' of method object at 0x00000227EB697900>